In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 

In [2]:
#Parameter
model = AbstractModel()
model.gen=Set()
model.t=Set()
model.a=Param(model.gen)
model.b=Param(model.gen)
model.c=Param(model.gen)
model.d=Param(model.gen)
model.e=Param(model.gen)
model.f=Param(model.gen)
model.Pmin=Param(model.gen)
model.Pmax=Param(model.gen)
model.RU=Param(model.gen)
model.RD=Param(model.gen)

model.teta=Param(model.t)

model.wind=Param(model.t)
model.demand=Param(model.t)

#Scalar
model.vwc=Param(initialize=50,mutable=True)


In [3]:
def Pbounds(model,gen,t):
  return (model.Pmin[gen],model.Pmax[gen])

def Pwbounds(model,t):
  return (0,model.wind[t])

model.p=Var(model.gen,model.t,bounds=Pbounds,within=Reals)
model.pw=Var(model.t,bounds=Pwbounds,within=Reals)
model.pwc=Var(model.t,bounds=Pwbounds,within=Reals)
model.cost=Var(within=Reals)
model.EM=Var(within=Reals)

def rule1(model,t):
  return sum(model.p[gen,t] for gen in model.gen) + model.pw[t] >= model.demand[t]
model.c1 = Constraint(model.t,rule=rule1)

def rule2(model,gen,t):
    if t==1:
      return model.p[gen,t+1] - model.p[gen,t] <= model.RU
    else:
        return Constraint.Skip
model.c2 = Constraint(model.gen,model.t,rule=rule2)

def rule3(model,gen,t):
    if t==1:
      return model.p[gen,t-1]- model.p[gen,t] <= model.RD
    else:
        return Constraint.Skip
model.c3 = Constraint(model.gen,model.t,rule=rule3)

def rule6(model,t):
  return model.pw[t] + model.pwc[t] == model.wind[t]
model.c6 = Constraint(model.t,rule=rule6)

def ruleTC(model):
  return model.cost == sum((model.a[gen]*model.p[gen,t]*model.p[gen,t] + 
                              model.b[gen]*model.p[gen,t]+model.c[gen]) for gen in model.gen for t in model.t) + sum(model.vwc + model.pwc[t] for t in model.t)
model.c4 = Constraint(rule=ruleTC)



In [4]:
model.obj=Objective(expr=model.cost,sense=minimize)

In [5]:
instance= model.create_instance("code44.dat")

In [7]:
results = SolverFactory('mindtpy').solve(instance,mip_solver='glpk', nlp_solver='ipopt')

In [8]:
instance.pprint()

2 Set Declarations
    gen : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'G1', 'G2', 'G3', 'G4'}
    t : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24'}

14 Param Declarations
    Pmax : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :   200
         G2 :   290
         G3 :   190
         G4 :   260
    Pmin : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :    28
         G2 :    20
         G3 :    30
         G4 :    20
    RD : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :    40
         G2 :    30
         G3 :    30
         G4 :    50

In [9]:
instance.cost.display()

cost : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :  None : 218967.1819923261 :  None : False : False :  Reals


In [11]:
instance.display()

Model unknown

  Variables:
    p : Size=96, Index=gen*t
        Key           : Lower : Value              : Upper : Fixed : Stale : Domain
         ('G1', 't1') :    28 :  88.38879020475889 :   200 : False : False :  Reals
        ('G1', 't10') :    28 : 139.76790999949935 :   200 : False : False :  Reals
        ('G1', 't11') :    28 : 106.87507400188771 :   200 : False : False :  Reals
        ('G1', 't12') :    28 : 119.01865008240651 :   200 : False : False :  Reals
        ('G1', 't13') :    28 :  88.54815472025149 :   200 : False : False :  Reals
        ('G1', 't14') :    28 :  60.40438128416702 :   200 : False : False :  Reals
        ('G1', 't15') :    28 :  49.75883164896644 :   200 : False : False :  Reals
        ('G1', 't16') :    28 :  54.63538582323519 :   200 : False : False :  Reals
        ('G1', 't17') :    28 :  76.05397670562243 :   200 : False : False :  Reals
        ('G1', 't18') :    28 : 127.43309650040038 :   200 : False : False :  Reals
        ('G1', 't19